# Question 1

In [68]:
import glob
import os
from datetime import date

import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col

In [ ]:
spark = SparkSession.builder.master("local[*]").appName("hw5").getOrCreate()

25/03/05 20:50:40 WARN Utils: Your hostname, tachyon resolves to a loopback address: 127.0.1.1; using 192.168.50.201 instead (on interface wlp4s0)
25/03/05 20:50:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/05 20:50:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.5.5'

# Question 2

In [ ]:
df = spark.read.parquet("./data/yellow_tripdata_2024-10.parquet")

In [5]:
df_part = df.repartition(4)

In [6]:
df_part.write.parquet("data/partitioned/")

In [10]:
files = glob.glob("data/partitioned/*.parquet")

In [18]:
total_size = 0.0
for file in files:
    total_size += os.path.getsize(file)
avg_size = total_size / len(files) / 1000**2
print(f"The average size of the created files is {np.round(avg_size)} MB")

The average size of the created files is 23.0 MB


In [20]:
df_part.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



# Question 3

In [53]:
df_part.filter(
    F.to_date(df_part.tpep_pickup_datetime) == date(year=2024, month=10, day=15)
).count()

128893

# Question 4

In [71]:
df_part.withColumn(
    "trip_duration",
    (
        F.unix_timestamp(df_part.tpep_dropoff_datetime)
        - F.unix_timestamp(df_part.tpep_pickup_datetime)
    )
    / 3600,
).select("trip_duration").sort(col("trip_duration").desc()).show()

+------------------+
|     trip_duration|
+------------------+
|162.61777777777777|
|           143.325|
|137.76055555555556|
|114.83472222222223|
| 89.89833333333333|
| 89.44611111111111|
| 70.29916666666666|
| 67.57333333333334|
| 66.06666666666666|
|           46.4225|
| 42.30888888888889|
| 38.47416666666667|
| 33.95111111111111|
| 26.29861111111111|
| 25.29138888888889|
|25.238333333333333|
|             24.47|
|23.996666666666666|
|23.995277777777776|
|23.994722222222222|
+------------------+
only showing top 20 rows



# Question 5
The user interface appears on port 4040

# Question 6

In [72]:
df_zones = spark.read.option("header", "true").csv("./data/taxi_zone_lookup.csv")

In [73]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [85]:
df_join = (
    df_part.withColumn("LocationID", col("PULocationID"))
    .join(df_zones, on="LocationID", how="inner")
    .groupBy("Zone")
    .count()
    .sort(col("count").asc())
)

In [86]:
df_join.show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|Governor's Island...|    1|
|       Rikers Island|    2|
|       Arden Heights|    2|
|         Jamaica Bay|    3|
| Green-Wood Cemetery|    3|
|Charleston/Totten...|    4|
|   Rossville/Woodrow|    4|
|       Port Richmond|    4|
|Eltingville/Annad...|    4|
|       West Brighton|    4|
|         Great Kills|    6|
|        Crotona Park|    6|
|Heartland Village...|    7|
|     Mariners Harbor|    7|
|Saint George/New ...|    9|
|             Oakwood|    9|
|       Broad Channel|   10|
|New Dorp/Midland ...|   10|
|         Westerleigh|   12|
|     Pelham Bay Park|   12|
+--------------------+-----+
only showing top 20 rows

